Preamble

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import scipy.stats as stats
import seaborn as sns

import statsmodels.api as sm



# for regression
import statsmodels.formula.api as smf
from sklearn import datasets
from statistics import mean

Load mdbtools framework

In [11]:
!sudo apt install mdbtools  

# ways that didn't work
# 1) !apt install unixodbc-dev  
#    !pip install pyodbc # this couldn't find my database file
# 2)  # !pip install mdbtools  # this installs a different version of mdbtools that doesn't work
# 3)  # !pip install meza   # meza is another way to read mdb files

Reading package lists... Done
Building dependency tree       
Reading state information... Done
mdbtools is already the newest version (0.7.1-6).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [12]:
import subprocess # from mdbtools
 
def show_tables(path='class_mtcars.mdb'):
    tables = subprocess.check_output(["mdb-tables", path])
    return tables.decode().split()

a = show_tables()
print(a)

['mtcars']


In [13]:
myFileName = 'class_mtcars.mdb'
myTable = 'mtcars'

def show_data(path=myFileName, table=myTable):
    tables = subprocess.check_output(["mdb-export", path, table])
    return tables.decode().split('\n')
 
def convert_df(path, table):
    d = show_data(path, table)
    columns = d[0].split(',')
    data = [i.split(',') for i in d[1:]]
    df = pd.DataFrame(columns=columns, data=data)
    return df

Load data

In [14]:
new_df = convert_df(path='class_mtcars.mdb', table='mtcars')

new_df.columns

# new_df.head

Index(['ID', 'model', 'mpg', 'cyl', 'disp', 'hp', 'drat', 'wt', 'qsec', 'vs',
       'am', 'gear', 'carb'],
      dtype='object')

Trim "model" column, convert the rest to float

In [15]:
new_df = new_df.drop(columns = 'model') 
print(new_df.columns)

print(new_df.dtypes)

new_df = new_df.apply(pd.to_numeric)

print(new_df.dtypes)


Index(['ID', 'mpg', 'cyl', 'disp', 'hp', 'drat', 'wt', 'qsec', 'vs', 'am',
       'gear', 'carb'],
      dtype='object')
ID      object
mpg     object
cyl     object
disp    object
hp      object
drat    object
wt      object
qsec    object
vs      object
am      object
gear    object
carb    object
dtype: object
ID      float64
mpg     float64
cyl     float64
disp    float64
hp      float64
drat    float64
wt      float64
qsec    float64
vs      float64
am      float64
gear    float64
carb    float64
dtype: object


DP

In [16]:

print(new_df.shape)
print(list(new_df.columns))

X = new_df.loc[:, new_df.columns != 'mpg' ]
# print(list(X.columns))
y = new_df.loc[:, new_df.columns == 'mpg']
X=sm.add_constant(X)

y

print(X.iloc[3,3])
print(type(X.iloc[3,3]))
print(type(y))

(33, 12)
['ID', 'mpg', 'cyl', 'disp', 'hp', 'drat', 'wt', 'qsec', 'vs', 'am', 'gear', 'carb']
258.0
<class 'numpy.float64'>
<class 'pandas.core.frame.DataFrame'>


Model using OLS regression

In [17]:
model = smf.ols('mpg ~ cyl + disp', data=new_df).fit()

# model = smf.ols('y ~ M + Ed + Po1 + Wealth + Ineq + Prob ', data=data).fit()

predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.760
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     45.81
Date:                Fri, 23 Oct 2020   Prob (F-statistic):           1.06e-09
Time:                        21:54:37   Log-Likelihood:                -79.573
No. Observations:                  32   AIC:                             165.1
Df Residuals:                      29   BIC:                             169.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     34.6610      2.547     13.609      0.0